# Categorical Naive Bayes

In [37]:
import numpy as np
from pathlib import Path

from src.models import CategoricalNaiveBayes
from src.utils import train_test_split, classification_report

# Data Prep

## Read data

In [38]:
data_path = Path("./data/processed/processed_train.csv")

# Read first line as header
with open(data_path, "r", encoding="utf-8") as f:
    data_header = f.readline().strip().split(",")

data = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str)
data[:5]

array([['city_103', 'Male', 'Has relevent experience', 'no_enrollment',
        'Graduate', 'STEM', '>20', 'Missing', 'Missing', '1', '1.0'],
       ['city_40', 'Male', 'No relevent experience', 'no_enrollment',
        'Graduate', 'STEM', '15', '50-99', 'Pvt Ltd', '>4', '0.0'],
       ['city_21', 'Missing', 'No relevent experience',
        'Full time course', 'Graduate', 'STEM', '5', 'Missing',
        'Missing', 'never', '0.0'],
       ['city_115', 'Missing', 'No relevent experience', 'no_enrollment',
        'Graduate', 'Business Degree', '<1', 'Missing', 'Pvt Ltd',
        'never', '1.0'],
       ['city_162', 'Male', 'Has relevent experience', 'no_enrollment',
        'Masters', 'STEM', '>20', '50-99', 'Funded Startup', '4', '0.0']],
      dtype='<U23')

## Split data

80% train, 20% val

In [39]:
X_train, X_val, y_train, y_val = train_test_split(
    data[:, :-1], data[:, -1], test_size=0.2, random_state=42
)

# Modeling

In [40]:
model = CategoricalNaiveBayes()
model.fit(X_train, y_train)

In [41]:
y_pred = model.predict(X_val)

In [42]:
classification_report(y_val, y_pred)

Class 0.0: Precision: 0.7444, Recall: 0.7291, F1-Score: 0.7367
Class 1.0: Precision: 0.7346, Recall: 0.7497, F1-Score: 0.7420
Overall Accuracy: 0.7394


{np.str_('0.0'): {'precision': np.float64(0.744408945686901),
  'recall': np.float64(0.7291376912378303),
  'f1_score': np.float64(0.7366941858422624)},
 np.str_('1.0'): {'precision': np.float64(0.7345826235093696),
  'recall': np.float64(0.7496522948539638),
  'f1_score': np.float64(0.7420409568060574)},
 'accuracy': np.float64(0.7393949930458971)}

## Predict test set

In [43]:
data_path = Path("./data/processed/processed_test.csv")

# Read first line as header
with open(data_path, "r", encoding="utf-8") as f:
    data_header = f.readline().strip().split(",")

data = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str)
data[:5]

array([['32403', 'city_41', 'Male', 'Has relevent experience',
        'Full time course', 'Graduate', 'STEM', '9', '<10', 'Missing',
        '1'],
       ['9858', 'city_103', 'Female', 'Has relevent experience',
        'no_enrollment', 'Graduate', 'STEM', '5', 'Missing', 'Pvt Ltd',
        '1'],
       ['31806', 'city_21', 'Male', 'No relevent experience',
        'no_enrollment', 'High School', 'STEM', '<1', 'Missing',
        'Pvt Ltd', 'never'],
       ['27385', 'city_13', 'Male', 'Has relevent experience',
        'no_enrollment', 'Masters', 'STEM', '11', '10/49', 'Pvt Ltd',
        '1'],
       ['27724', 'city_103', 'Male', 'Has relevent experience',
        'no_enrollment', 'Graduate', 'STEM', '>20', '10000+', 'Pvt Ltd',
        '>4']], dtype='<U23')

In [44]:
# save enrollee ids for submission
enrollee_ids = data[:, data_header.index("enrollee_id")]
X_test = data[:, :-1]
y_test_pred = model.predict(X_test)

In [45]:
# save predictions to csv file
submission_path = Path("./data/processed/submission.csv")
with open(submission_path, "w", encoding="utf-8") as f:
    f.write("enrollee_id,job_change\n")
    for enrollee_id, pred in zip(enrollee_ids, y_test_pred):
        f.write(f"{enrollee_id},{pred}\n")